# **Install OpenAI Agents Dep.**

In [ ]:
!pip install -Uq openai-agents

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# **Provider Config**

In [ ]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [ ]:
BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.0-flash-exp:free"

# **1. Using the OpenRouter API directly**

In [ ]:
import requests
import json

response = requests.post(
    url=f"{BASE_URL}/chat/completions",
    headers={
        "Authorization" : f"Bearer {OPENROUTER_API_KEY}",
    },
    data=json.dumps({
      "model" : MODEL,
      "messages" : [
        {
          "role": "user",
          "content" : "What is the meaning of life?"
        }
        ]
    })
)

print(response.json())

In [ ]:
data = response.json()
data["choices"][0]["message"]["content"]

# **2. Using OpenAI Agents SDK**

In [ ]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())
